In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import albumentations as A
import cv2
import numpy as np
import skimage as ski

import matplotlib.pyplot as plt
import os
import copy

from tqdm import tqdm
from IPython.display import clear_output

import psutil
import pynvml
import sys

import struct

import sklearn.metrics as metrics

sys.path.append('/home/meribejayson/Desktop/Projects/SharkCNN/training_models/dataloaders/')

from test_dataset import SharkDatasetTest as SharkDataset

In [2]:
lr_model_path = "/home/meribejayson/Desktop/Projects/SharkCNN/training_models/LOGISTIC-REG/train-final-3/lr_weights_train_3.tar"
ann_model_path = "/home/meribejayson/Desktop/Projects/SharkCNN/training_models/ANN/train-1/ann_weights_train_1.tar"

state_dict_lr = torch.load(lr_model_path)
state_dict_ann = torch.load(ann_model_path)

torch.manual_seed(12)

if not torch.cuda.is_available():
    raise Exception("Couldn't find CUDA")

device = torch.device("cuda")

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)


image_width = 1920
image_height = 1080

target_iters = 300
images_per_iter = 5

target_sample = image_height * image_width * target_iters * images_per_iter

In [3]:
class ANN(nn.Module):

    def __init__(self, input_size):
        super().__init__()
        
        hidden_layer_num = int(input_size / 2)

        self.linear1 = nn.Linear(input_size, hidden_layer_num)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_layer_num + input_size, 1)
        self.sig = nn.Sigmoid()
        

    def forward(self, x):
        h = self.linear1(x)
        h = self.relu1(h)
        h = torch.cat((x, h), dim=1)
        h = self.linear2(h)
        
        return self.sig(h)

In [4]:
class LogisticRegresion(nn.Module):

    def __init__(self, input_size):
        super().__init__()
        
        self.linear = nn.Linear(input_size,1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        
        return self.sig(x)

In [5]:
test_dataset = SharkDataset(batch_size=images_per_iter)

test_data_loader = data.DataLoader(test_dataset, batch_size=500_000, num_workers=3)

In [6]:
ann_state = torch.load("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/ANN/train-4/ann_weights_train_4.tar")
lr_state = torch.load("/home/meribejayson/Desktop/Projects/SharkCNN/training_models/LOGISTIC-REG/train-final-6/lr_weights_train_6.tar")

ann_model = ANN(85)
ann_model.load_state_dict(ann_state)
ann_model.to(device)


lr_model = LogisticRegresion(85)
lr_model.load_state_dict(lr_state)
lr_model.to(device)


LogisticRegresion(
  (linear): Linear(in_features=85, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [7]:
def append_to_binary_file(file_path, ann_preds, lr_preds, labels):
    with open(file_path, 'ab') as file:
        len = ann_preds.shape[0]

        for idx in range(len):
            packed_data = struct.pack('ffI', ann_preds[idx], lr_preds[idx], labels[idx])
            file.write(packed_data)

In [8]:
def read_from_binary_file(file_path):
    data = []
    with open(file_path, 'rb') as file:
        while True:
            packed_data = file.read(12)  # 4 bytes for float, 4 bytes for float, 4 bytes for int
            if not packed_data:
                break
            ann_pred, lr_pred, label = struct.unpack('ffI', packed_data)
            data.append((ann_pred, lr_pred, label))
    
    # Optionally convert to NumPy arrays
    ann_preds, lr_preds, labels = zip(*data)
    return np.array(ann_preds), np.array(lr_preds), np.array(labels).astype(int)

In [9]:
output_file_path = 'preds_labels.dat'

curr_idx = 0

for i in tqdm(range(target_iters)):
    with torch.no_grad():
        for point in test_data_loader:
            data_inputs = point[:, :-1].to(device).float()
            data_labels = point[:, -1].to(device).float()

            labels = data_labels.to('cpu').numpy().astype(int).flatten()
            
            ann_preds = ann_model(data_inputs).to('cpu').numpy().flatten()
            lr_preds = lr_model(data_inputs).to('cpu').numpy().flatten()


            append_to_binary_file(output_file_path, ann_preds, lr_preds, labels)


100%|██████████| 300/300 [24:41:11<00:00, 296.24s/it]   


In [10]:
preds_ann, preds_lr, all_labels = read_from_binary_file(output_file_path)

In [ ]:
precision_ann, recall_ann, pr_thresh_ann = metrics.precision_recall_curve(all_labels, preds_ann)
precision_lr, recall_lr, pr_thresh_lr = metrics.precision_recall_curve(all_labels, preds_lr)

In [ ]:
pr_plot_ann = metrics.PrecisionRecallDisplay(precision=precision_ann, recall=recall_ann)
pr_plot_ann.plot()
plt.title('Precision-Recall ANN')
plt.show()
plt.clf()

pr_plot_lr = metrics.PrecisionRecallDisplay(precision=precision_lr, recall=recall_lr)
pr_plot_lr.plot()
plt.title('Precision-Recall LR')
plt.show()
plt.clf()

In [ ]:
fpr_ann, tpr_ann, tp_thresh_ann = metrics.roc_curve(all_labels, preds_ann)
fpr_lr, tpr_lr, tp_thresh_lr = metrics.roc_curve(all_labels, preds_lr)

In [ ]:
roc_plot_ann = metrics.RocCurveDisplay(fpr=fpr_ann, tpr=tpr_ann)
roc_plot_ann.plot()
plt.title('ROC ANN')
plt.show()
plt.clf()


roc_plot_lr = metrics.RocCurveDisplay(fpr=fpr_lr, tpr=tpr_lr)
roc_plot_lr.plot()
plt.title('ROC LR')
plt.show()
plt.clf()

In [ ]:
roc_auc_score_ann = metrics.roc_auc_score(all_labels, preds_ann)
roc_auc_score_lr = metrics.roc_auc_score(all_labels, preds_lr)

print(f"The area under the ANN ROC curve is {roc_auc_score_ann}\nThe area under the LR ROC curve is {roc_auc_score_lr}")

In [ ]:
f1_score_ann = (2 * precision_ann * recall_ann) / (precision_ann + recall_ann)
f1_score_lr = (2 * precision_lr * recall_lr) / (precision_lr + recall_lr)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(pr_thresh_ann, f1_score_ann, color='blue')

plt.xlabel('Confidence')
plt.ylabel('F1-Score')
plt.title('F1-Score ANN')
plt.show()

plt.clf()

plt.figure(figsize=(12, 8))
plt.plot(pr_thresh_lr, f1_score_lr, color='blue')


plt.xlabel('Confidence')
plt.ylabel('F1-Score')
plt.title('F1-Score LR')
plt.show()

plt.clf()

In [ ]:
confs = [0.5, 0.55, 0.60, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

map_50_ann = 0
map_50_95_ann = 0
map_50_lr = 0
map_50_95_lr = 0

map_temp_lr = 0

map_50_lr = np.mean(precision_lr[pr_thresh_lr > 0.5])
map_50_ann = np.mean(precision_ann[pr_thresh_ann > 0.5])

for conf in confs:
    map_50_95_lr += np.mean(precision_lr[pr_thresh_lr > conf])
    map_50_95_ann += np.mean(precision_ann[pr_thresh_ann > conf])

print("ANN:")
print(f"MAP50: {map_50_ann} MAP50-95: {map_50_95_ann}\n")

print("LR:")
print(f"MAP50: {map_50_lr} MAP50-95: {map_50_95_lr}")